# Рубежный контроль №2

## Чичикин Т.Д. ИУ5-25М

Решение задачи классификации текстов.
Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

В качестве классификаторов необходимо использовать два классификатора по варианту для Вашей группы:

Группа	|Классификатор №1|Классификатор №2
---|---|---
**ИУ5-25М**|**KNeighborsClassifier**|**Complement Naive Bayes (CNB)**

Для каждого метода необходимо оценить качество классификации. Сделайте вывод о том, какой вариант векторизации признаков в паре с каким классификатором показал лучшее качество.


In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [2]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [3]:
# Загрузка данных
df = pd.read_csv('D:\\Ботва\\Магистратура\\2сем\\ММО\\РК2\\Лосева\\imdb_sup.csv')
text_df=df.head(500).append(df.tail(500))
text_df.drop('Rating', axis=1, inplace=True)
text_df.head(15)

,Review,Sentiment
0,"Kurt Russell's chameleon-like performance, cou...",1
1,It was extremely low budget(it some scenes it ...,1
2,James Cagney is best known for his tough chara...,1
3,"Following the brilliant ""Goyôkiba"" (aka. ""Hanz...",1
4,One of the last classics of the French New Wav...,1
5,Having just watched this film again from a 199...,1
6,The Straight Story is a truly beautiful movie ...,1
7,Four teenage girlfriends drive to Fort Laurdal...,1
8,"I haven't seen all of Jess Franco's movies, I ...",1
9,What's in a name? If the name is Jerry Bruckhe...,1


Изначально датасет содержит 50000 строк, что для выполнения нашей задачи слишком много. Так что сделаем из него датасет по-меньше

In [4]:
text_df.shape

(1000, 2)

In [5]:
text_df['Sentiment'].unique()

array([1, 0], dtype=int64)

Наш целевой признак - столбец Sentiment, который имеет всего 2 значения: 1 - если комментарий о фильме был положительным и 0 - если комментарий о фильме был отрицательным

In [6]:
# Сформируем общий словарь для обучения моделей из обучающей и тестовой выборки
vocab_list = text_df['Review'].tolist()
vocab_list[1:10]

["It was extremely low budget(it some scenes it looks like they recorded with a home video recorder). However it does have a good plot line, and its easy to follow. 8 years after shooting her sexually abusive step father Amanda is released from the psychiatric ward, with the help of her doctor who she is secretly having an affair with. The doctor ends up renting her a house and buying her a car. But within the first 20 minutes of the movie Amanda kills him and buries him in her backyard. Then she see's her neighbor Richard sets eyes on him and stops at nothing until she has him. She acts innocent but after another neighbor Buzz finds out that Amanda killed that doctor and attempted to kill Richards wife Laurie (this is after Amanda and him get it on in the hot tub). Then she stops acting so Innocent and kills Buzz and later on attempts to kill Richard whom she supposedly loves and cares for. And you'll have to rent the movie to find out if Amanda dies or not. Overall good movie, remind

In [7]:
vocabVect = CountVectorizer()
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 17896


In [8]:
for i in list(corpusVocab)[0:10]:
    print('{}={}'.format(i, corpusVocab[i]))

kurt=9020
russell=13645
chameleon=2717
like=9354
performance=11707
coupled=3693
with=17607
john=8666
carpenter=2536
flawless=6190


# Векторизация признаков на основе CountVectorizer
Подсчитывает количество слов словаря, входящих в данный текст

In [9]:
test_features = vocabVect.transform(vocab_list)

In [10]:
test_features

<1000x17896 sparse matrix of type '<class 'numpy.int64'>'
	with 137926 stored elements in Compressed Sparse Row format>

In [11]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

17896

In [13]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 8,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

# Векторизация признаков на основе TfidfVectorizer
Вычисляет специфичность текста в корпусе текстов на основе метрики TF-IDF

In [14]:
tfidfv = TfidfVectorizer(ngram_range=(1,2))
tfidf_ngram_features = tfidfv.fit_transform(vocab_list)
tfidf_ngram_features

<1000x138585 sparse matrix of type '<class 'numpy.float64'>'
	with 351280 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_ngram_features.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

# Оценка качества классификации

In [16]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, text_df['Review'], text_df['Sentiment'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [17]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [ComplementNB(), KNeighborsClassifier()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0069': 2, '007': 3, '01': 4,
                            '06th': 5, '08': 6, '0f': 7, '10': 8, '100': 9,
                            '100th': 10, '101': 11, '102': 12, '10th': 13,
                            '11': 14, '112': 15, '11th': 16, '12': 17, '13': 18,
                            '13th': 19, '14': 20, '14th': 21, '15': 22,
                            '150': 23, '16': 24, '1600s': 25, '16éme': 26,
                            '17': 27, '1710': 28, '18': 29, ...})
Модель для классификации - ComplementNB()
Accuracy = 0.7990115864367362
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0069': 2, '007': 3, '01': 4,
                            '06th': 5, '08': 6, '0f': 7, '10': 8, '100': 9,
                            '100th': 10, '101': 11, '102': 12, '10th': 13,
                            '11': 14, '112': 15, '11th': 16, '12': 17, '13': 18,
                            '13th': 19, '14': 20, '14th': 21, '15':

Наибольшая точность получилась при использовании CountVectorizer и ComplementNB()